#### Optimization & Analytics 23/24

#### Names:

##### - Mohamed Afif Chifaoui (100452024)
##### - Ricardo Vazquez Alvarez (100417117)

---


# Network Optimization and Non-linear models HW2

## 1. Network problem

Our **network optimization** relies on the Max-Flow Problem.
Spain is known to be the **main** exporter in the world of olive oil. According to the source **, in the year 2019, Spain exported around 210 thousand of tons.

However, in order to export these quantities the neighboring countries of Spain only allow a certain amount of oil to be passed through their roads, denominated as *capacity*. On the other hand, each of the importer countires do require a minimum quantity of oil to keep this agreement with Spain.

Our goal is to find the maximum amount of flow(tons of oil) that can circulate between countries in order to export the amount of olive oil required by each country.



!['oil'](img/oil.jpg)

### MODEL FORMULATION

#### SETS

- C = Set of Countries (each one represent a **node**)
  
    - (**Destinations:** Ireland, Poland, Italy, Switzerland ; **In between:** France, Monaco, UK, Belgium, Netherlands, and Germany).
  
  
- P = Set of connections (connected countries for flow)
    

- M = Set of minimum amount of olive oil required by importer countries.



#### PARAMETERS

- Source = source node, in our case **SPAIN**
- Destination = destination node, in our case (Ireland, Poland, Italy, Switzerland)
- Maximum = amount of olive oil allowed to pass by the countries **in-between**
- Needs = amount of olive oil required by importer countries


#### VARIABLES

- $Flow_{ij}$ = flow between origin country *i* and destination country *j*


### OBJECTIVE

Objective function is to maximize the amount of olive oil arriving at the final nodes (importer countries). Therefore, it is the total sum of the flow in every pair of countries **(connections)** that arrive to a final destination.

$$\text{max} \;\; \sum_{i,j\in P}^{} Flow_{ij}, \forall j \in Destination$$


### CONSTRAINTS

- 1. Flow of oil in each pair of connections (connected countries should not be higher than maximum allowed)

        $\forall{i,j \in P}, flow_{ij} \leq maximum_{ij}$

- 2. Flow into a node must be equal to the flow out of a node except if we are in source or destination, to fulfill *Flow Conservation Rule*.
  


        $\forall{b \in C}, \sum_{i \in P} Flow_{ib}$==$ \sum_{j \in P} Flow_{bj}$ iff $b$!=$j \in Destination$ or $i \in Source$


- 3. We must ensure that that importer countries receive at least their required amount of oil pacted in the agreement.


        
        $\forall{b \in C}, \sum_{bj \in C} Flow_{bj} \geq Needs_j$








In [31]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
import pandas
import math


**NOTE: the amount for param needs (total amount of olive distributed for all countries) must be less than what Spain (exporter country) has.**

In [32]:
%%writefile olive.dat
set C := Spain Ireland Poland Italy Switzerland France Monaco UK Belgium Netherlands Germany;
set P := (Spain, France) (Spain, Italy) (France, Monaco) (France, Italy) (France, Switzerland) (France, Belgium) (France, UK) (Belgium, Netherlands) (Belgium, Germany) (France, Germany) (UK, Ireland) (Netherlands, Germany) (Germany, Poland);

param origin := Spain;


param maximum :=
Spain France 160
Spain Italy 50
France Monaco 20
France Italy 150
France Switzerland 40
France Belgium 40
France UK 50
Belgium Netherlands 30
Belgium Germany 40
France Germany 30
UK Ireland 50
Netherlands Germany 20
Germany Poland 60;

param needs :=
Ireland 50
Poland 45
Italy 100
Switzerland 15;



Overwriting olive.dat


In [33]:
%%writefile olive_exp.py
from pyomo.environ import * 

model = AbstractModel()

model.C = Set()

# Initialize the Points with the cartesian product of the countries we have.
model.P = Set(within = model.C*model.C)

# Spain is the starting point
model.origin = Param(within = model.C)


model.maximum = Param(model.P)

model.needs = Param(model.C)

model.flow = Var(model.P, within = NonNegativeReals)


# Objective Function

def objective_rule(model):

    obj_val = sum(model.flow[origin,dest] for origin,dest in model.P if dest =='Ireland' or dest =='Poland' or dest == 'Italy' or dest =='Switzerland')

    return obj_val

model.obj = Objective(rule = objective_rule, sense = maximize)


def allowed_quantity(model, m, n):
    return model.flow[m, n] <= model.maximum[m, n]

def flow_rule(model , k):
    # if it is a final destination - dont flow out . If it is a 
    if k in ["Italy", "Switzerland", "Ireland","Poland", value(model.origin)]:
        return Constraint.Skip
    
    # the inflow from one origin to a destination MUST equal the outflow from origin to destination.
    # calculating the inflow
    inflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if dest==k)
    # calculating the outflow.
    outflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if origin==k)

    return inflow==outflow

def requested_oil(model, i):
    if i in ["Italy", "Switzerland", "Ireland","Poland"]:
        return sum(model.flow[j, k] for (j,k) in model.P if k==i)>=model.needs[i]
    else:
        return Constraint.Skip


model.flow_rule = Constraint(model.C, rule = flow_rule)
model.allowed_quantity = Constraint(model.P, rule = allowed_quantity)
model.requested_oil = Constraint(model.C, rule = requested_oil)


Overwriting olive_exp.py


In [34]:
!pyomo solve  olive_exp.py  olive.dat --solver=glpk --summary 

!type results.yml

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.07] Applying solver
[    0.32] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 210.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    flow : Size=13, Index=P
        Key                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Belgium', 'Germany') :     0 :  40.0 :  None : False : False : NonNegativeReals
        ('Belgium', 'Netherlands') :     0 :   0.0 :  None : False : False : NonNegativeReals
             ('France', 'Belgium') :     0 :  40.0 :  None : False : False : NonNegativeReals
             ('France', 'Germany') :     0 :   5.0 :  None : False : False : NonNegativeReals
               ('France', 'Italy') :     0 :  50.0 :  None : False : False : NonNegativeReals
              ('France', 'Monaco') :     0 : 

#### ANALYSIS OF RESULTS

> Obtained flows

In [35]:
import yaml
with open('results.yml') as f:
    doc = yaml.load(f,Loader=yaml.FullLoader)
    l1 = doc["Solution"][1]["Variable"]
    l1 = list(l1.items())
    
for i in l1:
    print(i[1]["Value"])

40
40
5
50
15
50
45
160
50
50


| Route | Max Flow solution | Maximum
| --- | --- | --- |
| Spain -> France | 160 | 160 |
| Spain -> Italy | 50 | 50 |
| France -> Belgium | 40 | 40 |
| France -> Germany | 5 | 30 |
| France -> Italy | 50 | 150 |
| France -> Switzerland | 15 | 40 |
| France -> UK | 50 | 50 |
| Belgium -> Germany | 40 | 40 |
| Germany -> Poland | 45 | 60 |
| UK -> Ireland | 50 | 50 |


## 2. Non-linear problem

# Bibliography




https://oec.world/es/profile/bilateral-product/olive-oil-virgin/reporter/pol